### Trabalho realizado pelo grupo G18

- #### José Silva A100105
- #### Alexandra Calafate A100060


# TP3 - Problema 1:


O algoritmo estendido de Euclides (EXA) aceita dois inteiros constantes a,b>0 e devolve inteiros r,s,t tais que a _ s + b _ t = r e r = gcd(a,b).

Para além das variáveis r,s,t o código requer 3 variáveis adicionais r',s',t' que representam os valores de r,s,t no “próximo estado”.


INPUT a, b

assume a > 0 and b > 0

r, r', s, s', t, t' = a, b, 1, 0, 0, 1

while r' != 0

q = r div r'

r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t'

OUTPUT r, s, t


1. Construa um SFOTS usando BitVector’s de tamanho n que descreva o comportamento deste programa. Considere estado de erro quando r=0 ou alguma das variáveis atinge o “overflow”.

2. Prove, usando a metodologia dos invariantes e interpolantes, que o modelo nunca atinge o estado de erro.


In [ ]:
"""
Implementação do Algoritmo de Euclides com Verificação Formal usando SMT
Este código implementa e verifica formalmente o algoritmo de Euclides para encontrar o MDC
usando verificação formal através da biblioteca pySMT.

Principais componentes:
1. Sistema de transição de estados (SFOTS - Symbolic Finite-state Open Transition System)
2. Verificação de invariantes
3. Prova formal de correção
"""

from pysmt.shortcuts import (Symbol, BVAdd, BVSub, BVMul, BVUDiv, BV, 
                           EqualsOrIff, And, Or, Not, BVULE, BVUGT, BVULT,
                           is_valid, Implies, BVUGE)
from pysmt.typing import BVType

def create_euclides_sfots(n):
    """
    Cria um sistema de transição simbólico (SFOTS) para o algoritmo de Euclides.
    
    Args:
        n (int): Número de bits para representação dos números
    
    Returns:
        dict: Dicionário contendo as variáveis, estado inicial, transições e estados de erro
    """

    # Declaração das variáveis de estado
    # r, r_linha: registradores principais para o algoritmo
    # s, s_linha, t, t_linha: variáveis auxiliares para coeficientes de Bézout
    r = Symbol("r", BVType(n))
    r_linha = Symbol("r_linha", BVType(n))
    s = Symbol("s", BVType(n))
    s_linha = Symbol("s_linha", BVType(n))
    t = Symbol("t", BVType(n))
    t_linha = Symbol("t_linha", BVType(n))
    
    # Variáveis para o próximo estado (usado nas transições)
    r_next = Symbol("r_next", BVType(n))
    r_linha_next = Symbol("r_linha_next", BVType(n))
    s_next = Symbol("s_next", BVType(n))
    s_linha_next = Symbol("s_linha_next", BVType(n))
    t_next = Symbol("t_next", BVType(n))
    t_linha_next = Symbol("t_linha_next", BVType(n))
    
    # Entradas do sistema: os números para calcular o MDC
    a = Symbol("a", BVType(n))
    b = Symbol("b", BVType(n))
    
    # Estado inicial com garantias de segurança
    init = And([
        # Garantias que os números são positivos e dentro dos limites seguros
        BVUGT(a, BV(0, n)),  # a > 0
        BVUGT(b, BV(0, n)),  # b > 0
        BVULE(a, BV(2**(n-2), n)),  # a <= 2^(n-2)
        BVULE(b, BV(2**(n-2), n)),  # b <= 2^(n-2)
        
        # Lógica para garantir que começamos com o maior número em r
        Implies(BVUGT(b, a),  # Se b > a
                And([
                    EqualsOrIff(r, b),      # r = b
                    EqualsOrIff(r_linha, a)  # r' = a
                ])),
        Implies(BVUGE(a, b),  # Se a >= b
                And([
                    EqualsOrIff(r, a),      # r = a
                    EqualsOrIff(r_linha, b)  # r' = b
                ])),
        
        # Inicialização dos coeficientes de Bézout
        EqualsOrIff(s, BV(1, n)),
        EqualsOrIff(s_linha, BV(0, n)),
        EqualsOrIff(t, BV(0, n)),
        EqualsOrIff(t_linha, BV(1, n))
    ])
    
    # Quociente para divisão
    q = Symbol("q", BVType(n))
    
    # Regras de transição do sistema
    trans = And([
        # Garantia que o divisor não é zero
        Not(EqualsOrIff(r_linha, BV(0, n))),
        
        # Cálculo do quociente e verificação
        EqualsOrIff(q, BVUDiv(r, r_linha)),
        BVULE(BVMul(q, r_linha), r),
        
        # Atualizações do estado seguindo o algoritmo de Euclides
        EqualsOrIff(r_next, r_linha),
        EqualsOrIff(r_linha_next, BVSub(r, BVMul(q, r_linha))),
        BVULE(r_linha_next, r_linha),
        
        # Atualização dos coeficientes de Bézout
        EqualsOrIff(s_next, s_linha),
        EqualsOrIff(s_linha_next, BVSub(s, BVMul(q, s_linha))),
        EqualsOrIff(t_next, t_linha),
        EqualsOrIff(t_linha_next, BVSub(t, BVMul(q, t_linha)))
    ])
    
    # Estados de erro (overflow ou divisão por zero)
    error = Or([
        EqualsOrIff(r, BV(0, n)),
        BVUGT(r, BV(2**(n-1), n))
    ])
    
    return {
        'vars': {
            'current': {'r': r, 'r_linha': r_linha, 's': s, 's_linha': s_linha, 
                       't': t, 't_linha': t_linha},
            'next': {'r': r_next, 'r_linha': r_linha_next, 's': s_next, 's_linha': s_linha_next,
                    't': t_next, 't_linha': t_linha_next},
            'input': {'a': a, 'b': b}
        },
        'init': init,
        'trans': trans,
        'error': error
    }

def verify_invariants(sfots, n):
    """
    Verifica os invariantes do sistema.
    
    Args:
        sfots (dict): Sistema de transição
        n (int): Número de bits
    
    Returns:
        dict: Resultados das verificações dos invariantes
    """
    current_vars = sfots['vars']['current']
    next_vars = sfots['vars']['next']
    
    r = current_vars['r']
    r_linha = current_vars['r_linha']
    
    # Definição dos invariantes
    inv1 = BVUGT(r, BV(0, n))          # r > 0
    inv2 = BVULE(r, BV(2**(n-1), n))   # r <= 2^(n-1)
    inv3 = BVULE(r_linha, r)           # r' <= r
    
    invariant = And(inv1, inv2, inv3)
    
    # Substitui as variáveis para verificação do próximo estado
    next_state_inv = invariant.substitute({
        current_vars['r']: next_vars['r'],
        current_vars['r_linha']: next_vars['r_linha']
    })
    
    # Verificações formais
    init_implies_inv = is_valid(Implies(sfots['init'], invariant))
    inv_implies_not_error = is_valid(Implies(invariant, Not(sfots['error'])))
    trans_preserves_inv = is_valid(Implies(And(invariant, sfots['trans']), next_state_inv))
    
    return {
        'init_implies_inv': init_implies_inv,
        'inv_implies_not_error': inv_implies_not_error,
        'trans_preserves_inv': trans_preserves_inv,
        'invariant': invariant
    }

# Execução do teste
def run_verification(n=8):
    """
    Executa a verificação do sistema com uma visualização mais clara dos resultados.
    
    Args:
        n (int): Número de bits para a verificação (padrão: 8)
    """
    print(f"Verificação do Algoritmo de Euclides ({n} bits)")
    
    sfots = create_euclides_sfots(n)
    results = verify_invariants(sfots, n)
    
    verification_results = {
        'init_implies_inv': 'O estado inicial satisfaz os invariantes',
        'inv_implies_not_error': 'Os invariantes garantem ausência de erros',
        'trans_preserves_inv': 'As transições preservam os invariantes'
    }
    
    for key, description in verification_results.items():
        status = "PASSOU" if results[key] else "FALHOU"
        print(f"\n{description}:")
        print(f"Status: {status}")
    
# Executar a verificação
if __name__ == "__main__":
    run_verification(8)